In [176]:
import pandas as pd

In [177]:
counties = pd.read_csv('counties-table.csv')

In [178]:
counties['state'].unique()

array(['California', 'Illinois', 'Texas', 'Arizona', 'Florida',
       'New York', 'Nevada', 'Washington', 'Michigan', 'Massachusetts',
       'Pennsylvania', 'Ohio', 'Minnesota', 'North Carolina', 'Utah',
       'Virginia', 'Georgia', 'Maryland', 'Hawaii', 'Missouri',
       'Connecticut', 'Indiana', 'New Jersey', 'Wisconsin', 'Tennessee',
       'Oklahoma', 'Oregon', 'Kentucky', 'Colorado',
       'District of Columbia', 'New Mexico', 'Rhode Island', 'Alabama',
       'Kansas', 'Nebraska', 'Delaware', 'South Carolina', 'Idaho',
       'Iowa', 'Louisiana', 'New Hampshire', 'Arkansas', 'Maine',
       'Alaska', 'Mississippi', 'South Dakota', 'North Dakota',
       'West Virginia'], dtype=object)

In [179]:
counties.describe()

,year,fips,pop2020,pop2024,growthSince2020,landArea,density
count,400.0,400.000000,4.000000e+02,4.000000e+02,400.000000,400.000000,400.000000
mean,2024.0,28444.545000,5.954469e+05,6.037931e+05,0.029064,1085.785000,1513.210063
std,0.0,15758.254022,7.750317e+05,7.616020e+05,0.054329,1723.504454,4626.409744
min,2024.0,1003.000000,1.471400e+05,1.736160e+05,-0.088976,23.000000,16.986124
25%,2024.0,13055.500000,2.314202e+05,2.387765e+05,-0.004377,441.500000,339.373465
50%,2024.0,29086.000000,3.649050e+05,3.785305e+05,0.015965,656.500000,571.363289
75%,2024.0,42032.000000,6.653718e+05,6.874150e+05,0.049208,941.750000,1250.843382
max,2024.0,55133.000000,9.992813e+06,9.606925e+06,0.350884,20068.000000,69580.826087


In [180]:
indiana_df = counties[counties['state'] == 'Indiana']


In [181]:
indiana_df

,year,fips,pop2020,pop2024,state,stateCode,growthSince2020,name,county,slug,landArea,density
53,2024,18097,976659,967697,Indiana,IN,-0.009176,Marion County,Marion County,indiana/marion-county,396,2443.679293
146,2024,18089,498957,501539,Indiana,IN,0.005175,Lake County,Lake County,indiana/lake-county,499,1005.088176
190,2024,18003,386103,397446,Indiana,IN,0.029378,Allen County,Allen County,indiana/allen-county,657,604.940639
200,2024,18057,348996,377827,Indiana,IN,0.082611,Hamilton County,Hamilton County,indiana/hamilton-county,394,958.951777
265,2024,18141,272682,273414,Indiana,IN,0.002684,St. Joseph County,St. Joseph County,indiana/st-joseph-county,458,596.973799
344,2024,18039,206991,205945,Indiana,IN,-0.005053,Elkhart County,Elkhart County,indiana/elkhart-county,463,444.805616
369,2024,18063,175596,189655,Indiana,IN,0.080064,Hendricks County,Hendricks County,indiana/hendricks-county,407,465.982801
370,2024,18157,186387,189347,Indiana,IN,0.015881,Tippecanoe County,Tippecanoe County,indiana/tippecanoe-county,499,379.452906
383,2024,18163,180336,180057,Indiana,IN,-0.001547,Vanderburgh County,Vanderburgh County,indiana/vanderburgh-county,233,772.776824
394,2024,18127,173318,175822,Indiana,IN,0.014447,Porter County,Porter County,indiana/porter-county,418,420.626794


In [182]:
indiana_df.describe()

,year,fips,pop2020,pop2024,growthSince2020,landArea,density
count,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,2024.0,18093.600000,340602.500000,345874.900000,0.021446,442.400000,809.327862
std,0.0,53.433656,249205.183846,246018.227749,0.033509,107.144969,615.238136
min,2024.0,18003.000000,173318.000000,175822.000000,-0.009176,233.000000,379.452906
25%,2024.0,18058.500000,181848.750000,189424.000000,-0.000489,398.750000,450.099912
50%,2024.0,18093.000000,239836.500000,239679.500000,0.009811,438.000000,600.957219
75%,2024.0,18137.500000,376826.250000,392541.250000,0.026004,490.000000,912.408038
max,2024.0,18163.000000,976659.000000,967697.000000,0.082611,657.000000,2443.679293


In [183]:
import pandas as pd
import re

# File path
file_path = 'AdjacentCounties'

# Reading the file
with open(file_path, 'r') as file:
    data = file.read()

# Initialize an empty list to store the processed data
county_data = []
current_county = None
current_code = None

# Process each line in the file data
for line in data.strip().splitlines():
    line = line.strip()
    if not line:
        continue

    # Match for main county lines (non-indented) using regex for pattern consistency
    match_main = re.match(r'"(.+?), IN"\s+(\d+)\s+".+?"\s+(\d+)', line)
    if match_main:
        # Extract the main county details only if it is an Indiana county
        current_county = match_main.group(1)
        current_code = match_main.group(2)
    else:
        # Process indented lines for adjacent counties
        match_adjacent = re.match(r'"\s*(.+?)"\s+(\d+)', line)
        if match_adjacent and current_county and current_code:
            adjacent_county = match_adjacent.group(1)
            adjacent_code = match_adjacent.group(2)
            county_data.append([current_county, current_code, adjacent_county, adjacent_code])

# Create DataFrame with Indiana counties only
df = pd.DataFrame(county_data, columns=["County", "County_Code", "Adjacent_County", "Adjacent_County_Code"])

# Get DataFrame with Indiana counties only in the Adjacent_County column
df = df[df["Adjacent_County"].str.endswith(", IN")]
df["Adjacent_County"] = df["Adjacent_County"].str.replace(", IN", "")

In [184]:
adjacent = df

In [185]:
adjacent.head()

,County,County_Code,Adjacent_County,Adjacent_County_Code
0,Adams County,18001,Allen County,18003
1,Adams County,18001,Jay County,18075
2,Adams County,18001,Wells County,18179
5,Allen County,18003,Allen County,18003
6,Allen County,18003,DeKalb County,18033


In [186]:
# Load populations
# Reading the data from the specified file 'county_data.csv'

# File path
file_path = 'county_data.csv'

# Reading the data from the file
pop_df = pd.read_csv(file_path)

# Add ' County' to end of name column
pop_df['name'] = pop_df['name'] + ' County'

pop_df.head()

,id,name,name_ascii,type,county_fips,state_id,state_name,lat,lng,population,metadata_tag
0,1,Whitley County,Whitley,County,18183,IN,Indiana,41.1394,-85.5051,34048,0
1,2,White County,White,County,18181,IN,Indiana,40.7498,-86.8655,24593,0
2,3,Wells County,Wells,County,18179,IN,Indiana,40.7292,-85.2212,28103,0
3,4,Wayne County,Wayne,County,18177,IN,Indiana,39.8644,-85.0098,66588,0
4,5,Washington County,Washington,County,18175,IN,Indiana,38.6000,-86.1053,28025,0


In [187]:
# Sum of all populations divide by 9 (number of counties in Indiana)
ideal_population = pop_df['population'].sum() / 9

# Calculate the min and max based on 5% tolerance
min_population = ideal_population - (ideal_population * 0.05)
max_population = ideal_population + (ideal_population * 0.05)

ideal_population, min_population, max_population

(750148.8888888889, 712641.4444444444, 787656.3333333334)

In [188]:
import pandas as pd

# Merge to add main county data (lat, lng, population)
adjacent = pd.merge(
    adjacent, 
    pop_df[['name', 'lat', 'lng', 'population']], 
    left_on='County', 
    right_on='name', 
    how='left'
).rename(columns={'lat': 'main_lat', 'lng': 'main_lng', 'population': 'main_population'})

# Drop the extra 'name' column
adjacent = adjacent.drop(columns=['name'])

# Merge again to add adjacent county data (lat, lng, population)
adjacent = pd.merge(
    adjacent, 
    pop_df[['name', 'lat', 'lng', 'population']], 
    left_on='Adjacent_County', 
    right_on='name', 
    how='left'
).rename(columns={'lat': 'adj_lat', 'lng': 'adj_lng', 'population': 'adj_population'})

# Drop the extra 'name' column from the second merge
adjacent = adjacent.drop(columns=['name'])

# Display the updated DataFrame
adjacent.head()


,County,County_Code,Adjacent_County,Adjacent_County_Code,main_lat,main_lng,main_population,adj_lat,adj_lng,adj_population
0,Adams County,18001,Allen County,18003,40.7457,-84.9366,35685,41.0909,-85.0666,381839
1,Adams County,18001,Jay County,18075,40.7457,-84.9366,35685,40.4380,-85.0057,20570
2,Adams County,18001,Wells County,18179,40.7457,-84.9366,35685,40.7292,-85.2212,28103
3,Allen County,18003,Allen County,18003,41.0909,-85.0666,381839,41.0909,-85.0666,381839
4,Allen County,18003,DeKalb County,18033,41.0909,-85.0666,381839,41.3976,-84.9991,43059


In [189]:
from math import pi, sin, cos, asin, sqrt

def degrees_to_radians(x):
     return((pi/180)*x)
     
def lon_lat_distance_miles(lon_a,lat_a,lon_b,lat_b):
    radius_of_earth = 24872/(2*pi)
    c = sin((degrees_to_radians(lat_a) - \
    degrees_to_radians(lat_b))/2)**2 + \
    cos(degrees_to_radians(lat_a)) * \
    cos(degrees_to_radians(lat_b)) * \
    sin((degrees_to_radians(lon_a) - \
    degrees_to_radians(lon_b))/2)**2
    return(2 * radius_of_earth * (asin(sqrt(c))))    

def lon_lat_distance_meters (lon_a,lat_a,lon_b,lat_b):
    return(lon_lat_distance_miles(lon_a,lat_a,lon_b,lat_b) * 1609.34) 
    

adjacent['distance_miles'] = adjacent.apply(
    lambda row: lon_lat_distance_miles(row['main_lat'], row['main_lng'], row['adj_lat'], row['adj_lng']),
    axis=1
)

In [190]:
adjacent.head()

,County,County_Code,Adjacent_County,Adjacent_County_Code,main_lat,main_lng,main_population,adj_lat,adj_lng,adj_population,distance_miles
0,Adams County,18001,Allen County,18003,40.7457,-84.9366,35685,41.0909,-85.0666,381839,9.218758
1,Adams County,18001,Jay County,18075,40.7457,-84.9366,35685,40.4380,-85.0057,20570,5.124827
2,Adams County,18001,Wells County,18179,40.7457,-84.9366,35685,40.7292,-85.2212,28103,19.662941
3,Allen County,18003,Allen County,18003,41.0909,-85.0666,381839,41.0909,-85.0666,381839,0.000000
4,Allen County,18003,DeKalb County,18033,41.0909,-85.0666,381839,41.3976,-84.9991,43059,5.011402


# Build Linear Programming Problem



Our objective is to maximize compactness (minimize distance between counties in the same district). Our constraints are:

• Each county must be in exactly one district.  
• The population of each district must be within 5% of the ideal population (the mean of the total population divided by the number of districts). These are currently stored in ideal_population, min_population, max_population variables. 
• The number of districts must be 9 (the current number of districts in Indiana).  

In [196]:
# Work in Progress
# Import PuLP library
from pulp import LpVariable, LpBinary, LpProblem, LpMaximize, lpSum, LpStatus

# Create a binary variable to indicate if a county is assigned to a district
adjacent['is_assigned'] = LpVariable.dicts("is_assigned", adjacent.index, 0, 1, LpBinary)

# Create a binary variable to indicate if two counties are in the same district
adjacent['is_same_district'] = LpVariable.dicts("is_same_district", adjacent.index, 0, 1, LpBinary)

# Initialize the model
model = LpProblem("Indiana Counties", LpMaximize)

# Objective function: maximize compactness
model += lpSum(adjacent['distance_miles'] * adjacent['is_same_district'])

# Constraint: each county must be in exactly one district
for i, county in adjacent.iterrows():
    model += lpSum(adjacent.loc[(adjacent['County'] == county['County']) | (adjacent['Adjacent_County'] == county['County']), 'is_same_district']) == 1
    
# Constraint: population of each district must be within 5% of the ideal population
for i in range(9):
    model += lpSum(adjacent.loc[j, 'main_population'] * adjacent.loc[j, 'is_assigned'] for j in adjacent.index if j % 9 == i) >= min_population
    model += lpSum(adjacent.loc[j, 'main_population'] * adjacent.loc[j, 'is_assigned'] for j in adjacent.index if j % 9 == i) <= max_population
    
# Constraint: number of districts must be 9
model += lpSum(adjacent['is_assigned']) == 9

# Solve the model
model.solve()

# Display the status of the model
LpStatus[model.status]

# Display the optimized district assignments
adjacent['district'] = [county['is_assigned'].varValue for i, county in adjacent.iterrows()]

# Display the optimized district assignments
adjacent[['County', 'district']].sort_values('district')

/opt/anaconda3/lib/python3.11/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4_/vbtv64ld2lv61jz5x9sljn5h0000gn/T/1b3c8599b1064ae0a18378b6de1a7c85-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/4_/vbtv64ld2lv61jz5x9sljn5h0000gn/T/1b3c8599b1064ae0a18378b6de1a7c85-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 544 COLUMNS
At line 11180 RHS
At line 11720 BOUNDS
At line 12761 ENDATA
Problem MODEL has 539 rows, 1040 columns and 8114 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1271.5 - 0.00 seconds
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.02   (Wallclock seconds):       0.03



,County,district
0,Adams County,0.000000
356,Pulaski County,0.000000
355,Pulaski County,0.000000
354,Pulaski County,0.000000
353,Pulaski County,0.000000
...,...,...
269,Marion County,0.735029
270,Marion County,0.735029
101,Elkhart County,1.000000
244,Lake County,1.000000
